In [9]:
import cv2
import numpy as np

In [10]:
def calculate_color_distribution(frame, bins=32):
    height, width, channels = frame.shape
    histogram = np.zeros((bins, bins, bins))

    for i in range(height):
        for j in range(width):
            b = int(frame[i, j, 0] * bins / 256)
            g = int(frame[i, j, 1] * bins / 256)
            r = int(frame[i, j, 2] * bins / 256)
            histogram[b, g, r] += 1

    histogram /= (height * width)
    return histogram.flatten()

In [11]:
def bhattacharyya_coefficient(hist1, hist2):
    bc = 0.0
    for i in range(len(hist1)):
        bc += np.sqrt(hist1[i] * hist2[i])
    return bc

In [12]:
def shot_segmentation(video_path, k=5, eta=0.8, bins=32):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()

    prev_distributions = [calculate_color_distribution(frame)]
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_distribution = calculate_color_distribution(frame)
        if len(prev_distributions) >= k:
            prev_distributions.pop(0)

        combined_hist = np.mean(prev_distributions, axis=0)
        bc = bhattacharyya_coefficient(combined_hist, current_distribution)
        
        if bc < eta:
            print("Scene change detected.")
        
        prev_distributions.append(current_distribution)
    
    cap.release()

In [13]:
# Example usage:
video_path = "C:\\Users\\sahua\\Downloads\\EE722\\Assignment_3\\Input\\Q4_Amul_Commercial.mp4"
shot_segmentation(video_path, k=5, eta=0.8)

Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene chang

In [14]:
shot_segmentation(video_path, k=5, eta=0.8)
shot_segmentation(video_path, k=10, eta=0.8)
shot_segmentation(video_path, k=5, eta=0.6)
shot_segmentation(video_path, k=5, eta=0.9)

Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene change detected.
Scene chang

In [15]:
def mark_scene_changes(video_path, output_path, k=5, eta=0.8, bins=32):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    ret, frame = cap.read()

    prev_distributions = [calculate_color_distribution(frame)]
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1

        current_distribution = calculate_color_distribution(frame)
        if len(prev_distributions) >= k:
            prev_distributions.pop(0)

        combined_hist = np.mean(prev_distributions, axis=0)
        bc = bhattacharyya_coefficient(combined_hist, current_distribution)
        
        if bc < eta:
            cv2.rectangle(frame, (0, 0), (frame_width, frame_height), (0, 255, 0), thickness=10)
        
        out.write(frame)
        prev_distributions.append(current_distribution)
    
    cap.release()
    out.release()

In [18]:
output_path = "C:\\Users\\sahua\\Downloads\\EE722\\Assignment_3\\Input\\green_marked_video_s.mp4"

# Call the function to mark scene changes and save the modified video
mark_scene_changes(video_path, output_path, k=5, eta=0.8)